In [1]:
#Instalando o pyspark
!pip install pyspark

     |████████████████████████████████| 281.3 MB 33 kB/s 
     |████████████████████████████████| 198 kB 63.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=80ed494e075bc5be46a404c5f332f38f09f24476c6a3f95276dca7af90b48fa4
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
#Importando o PySpark SQL e Functions
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
#Criando conexão com o spark através da porta 4050
spark = (SparkSession.builder #Configura um local para contrução de aplicativos
        .master("local") #Diz quem é o nó mestre, nesse caso, a máquina local
        .appName("aprendendoPySpark") #Diz qual é o nome do aplicativo sendo criado
        .config("spark.ui.port", "4050") #Configura a porta a ser utilizada para acessar Spark via API
        .getOrCreate()) #Comando para iniciar essa sessão

In [4]:
#Conferindo conexão
spark

In [5]:
#Carregando um dataframe externo
df = (spark #Conexão estabelecida anteriormente com o Spark
       .read #Informa que um documento será lido
       .format("csv") #Informa o formato do documento
       .option("header", "true") #Inclui um cabeçalho no documento
       .option("inferschema", "true") #Traz o tipo dos dados da coluna
       .option("delimiter", ";") #Informa o tipo de separador do documento CSV
       .load("/content/drive/MyDrive/arquivo_geral.csv") #Faz o carregamento do documento
)

In [6]:
df.show()

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No

In [10]:
#Inserindo determinado valor em uma coluna quando a condição for verdadeira, caso contrário, insere-se outro valor. 
df2 = (df.withColumn("status",
          F.when(F.col("casosNovos") > 0, "casos novos") #when = Quando condição for verdadeira, adcione esse valor.
          .otherwise("sem casos novos"))) #otherwise = Caso contrário, adcione este outro valor.
df2.show()

+------+------+----------+----------+---------------+-----------+----------------+---------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|         status|
+------+------+----------+----------+---------------+-----------+----------------+---------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|sem casos novos|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|sem casos novos|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|sem casos novos|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|sem casos novos|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|sem casos novos|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|sem casos novos|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|sem casos novos|
| Norte|  

In [16]:
#Passando a quantidade de casos novos e concatenando com o texto "casosNovos"
df3 = (
    df2.withColumn("status", 
    F.when(F.col("casosNovos")> 0, F.concat(df2.casosNovos, F.lit(" casos ovos"))) 
    #Concatenando num de casos novos á mensagem de "casos novos"
    .otherwise("Sem casos novos")
      ))
df3.select(F.col("data"), F.col("casosNovos"), F.col("status")).show(300)

+----------+----------+---------------+
|      data|casosNovos|         status|
+----------+----------+---------------+
|2020-01-30|         0|Sem casos novos|
|2020-01-31|         0|Sem casos novos|
|2020-02-01|         0|Sem casos novos|
|2020-02-02|         0|Sem casos novos|
|2020-02-03|         0|Sem casos novos|
|2020-02-04|         0|Sem casos novos|
|2020-02-05|         0|Sem casos novos|
|2020-02-06|         0|Sem casos novos|
|2020-02-07|         0|Sem casos novos|
|2020-02-08|         0|Sem casos novos|
|2020-02-09|         0|Sem casos novos|
|2020-02-10|         0|Sem casos novos|
|2020-02-11|         0|Sem casos novos|
|2020-02-12|         0|Sem casos novos|
|2020-02-13|         0|Sem casos novos|
|2020-02-14|         0|Sem casos novos|
|2020-02-15|         0|Sem casos novos|
|2020-02-16|         0|Sem casos novos|
|2020-02-17|         0|Sem casos novos|
|2020-02-18|         0|Sem casos novos|
|2020-02-19|         0|Sem casos novos|
|2020-02-20|         0|Sem casos novos|


In [18]:
#Pegando partes de uma string com a função "substring"
#Sintaxe = F.substring(string, posição, tamanho)

df4 = (
    df.withColumn("dia", F.substring(F.col("data"), 9, 2)) #Cria uma coluna dia pegando a posição do dia em "data"
    .withColumn("mes", F.substring(F.col("data"), 6, 2)) #Cria uma coluna mes pegando a posição do mes em "data"
    .withColumn("ano", F.substring(F.col("data"), 1, 4)) #Cria uma coluna ano pegando a posição do ano em "data"
)
lista_esquema = ["regiao", "dia", "mes", "ano", "obitosAcumulados"] #Atribui a uma lista os valores que serão pegos em df4
df4.select(lista_esquema).show() #Mostra os valores do dataframes listados em "lista_esquema"

+------+---+---+----+----------------+
|regiao|dia|mes| ano|obitosAcumulados|
+------+---+---+----+----------------+
| Norte| 30| 01|2020|               0|
| Norte| 31| 01|2020|               0|
| Norte| 01| 02|2020|               0|
| Norte| 02| 02|2020|               0|
| Norte| 03| 02|2020|               0|
| Norte| 04| 02|2020|               0|
| Norte| 05| 02|2020|               0|
| Norte| 06| 02|2020|               0|
| Norte| 07| 02|2020|               0|
| Norte| 08| 02|2020|               0|
| Norte| 09| 02|2020|               0|
| Norte| 10| 02|2020|               0|
| Norte| 11| 02|2020|               0|
| Norte| 12| 02|2020|               0|
| Norte| 13| 02|2020|               0|
| Norte| 14| 02|2020|               0|
| Norte| 15| 02|2020|               0|
| Norte| 16| 02|2020|               0|
| Norte| 17| 02|2020|               0|
| Norte| 18| 02|2020|               0|
+------+---+---+----+----------------+
only showing top 20 rows

